<a href="https://colab.research.google.com/github/youssefkamil/-Collaborative-Filtering-Recommender-System-for-amazon-products/blob/main/Collaborative_Filtering_Recommender_System_for_amazon_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **products Recommender System**

<a  align=center><img src = "https://uploads-ssl.webflow.com/60bfd2b558c9eba77e06bf57/60f591a8e727a820ea67615d_amazon-recos-feat.png" width=1000> </a>

<h1 align=center><font size = 5>Collaborative Filtering</font></h1>

**Steps:**
- Get products dataframe (given data)
- Get users ratings dataframe (given data)
- Get input user ratings (user input/assumption)
- Learning the similarity weights (Pearson Correlation)
- Find the recommendations (user profile * original products categories)

## 1. Get products dataframe

In [ ]:
import pandas as pd
from scipy.stats import pearsonr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
product_df = pd.read_csv('/content/drive/MyDrive/Amazon_products_reviews.csv')
product_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",NaN,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [ ]:
product_df=product_df.drop_duplicates(subset=['name','reviews.username'])
product_df=product_df.fillna(0)
product_df

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",0,True,0.0,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,0.0,0.0,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",0,True,0.0,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,0.0,0.0,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",0,True,0.0,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,0.0,0.0,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",0,True,0.0,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,0.0,0.0,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",0,True,0.0,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,0.0,0.0,explore42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34654,AVpfiBlyLJeJML43-4Tp,0,B006GWO5WK,Amazon,"Computers/Tablets & Networking,Tablet & eBook ...",newamazonkindlefirehd9wpowerfastadaptercharger...,"Amazon Digital Services, Inc",2012-10-16T00:00:00Z,0,2015-08-13T00:00:00Z,0,0,0.0,0.0,1.0,http://www.amazon.com/Amazon-PowerFast-Adapter...,This is exactly like any other usb power charg...,Not Necessary,0.0,0.0,mir_
34656,AVpfiBlyLJeJML43-4Tp,0,B006GWO5WK,Amazon,"Computers/Tablets & Networking,Tablet & eBook ...",newamazonkindlefirehd9wpowerfastadaptercharger...,"Amazon Digital Services, Inc",2012-11-21T00:00:00Z,0,2015-09-02T00:00:00Z,0,0,0.0,0.0,1.0,http://www.amazon.com/Amazon-PowerFast-Adapter...,Amazon should include this charger with the Ki...,Should be included,0.0,0.0,Jonathan Stewart
34657,AVpfiBlyLJeJML43-4Tp,0,B006GWO5WK,Amazon,"Computers/Tablets & Networking,Tablet & eBook ...",newamazonkindlefirehd9wpowerfastadaptercharger...,"Amazon Digital Services, Inc",2012-10-19T00:00:00Z,0,2015-09-04T00:00:00Z,0,0,0.0,0.0,1.0,http://www.amazon.com/Amazon-PowerFast-Adapter...,Love my Kindle Fire but I am really disappoint...,Disappointing Charger,0.0,0.0,J Lawson
34658,AVpfiBlyLJeJML43-4Tp,0,B006GWO5WK,Amazon,"Computers/Tablets & Networking,Tablet & eBook ...",newamazonkindlefirehd9wpowerfastadaptercharger...,"Amazon Digital Services, Inc",2012-10-31T00:00:00Z,0,2015-09-01T00:00:00Z,

## 2. Get users ratings dataframe

In [ ]:
#Drop genres as we don't need them at Collaborative Filtering technique
product_df = product_df[['name','reviews.rating','reviews.username']]
product_df.head()

,name,reviews.rating,reviews.username
0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,Adapter
1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,truman
2,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,DaveZ
3,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Shacks
4,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,explore42


In [ ]:
product_df['reviews.rating'].unique()

array([5., 4., 2., 1., 3., 0.])

## 3. Get input user ratings

We will use __Collaborative Filtering recommendation system__, which is also known as __User-User Filtering__.

This technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input user and then recommends items that they have liked to him. There are several methods of finding similar users, and the one we will use here is based on the __Pearson Correlation Function__.

In [ ]:
userInput = [
            {'name':product_df['name'].unique()[0], 'rating':5},
            {'name':product_df['name'].unique()[10], 'rating':3.5},
            {'name':product_df['name'].unique()[20], 'rating':2},
            {'name':product_df['name'].unique()[30], 'rating':5},
            {'name':product_df['name'].unique()[40], 'rating':4.5}
         ] 
inputProducts = pd.DataFrame(userInput)
inputProducts

,name,rating
0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0
1,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",3.5
2,Amazon Kindle Paperwhite - eBook reader - 4 GB...,2.0
3,"Amazon - Fire 16GB (5th Gen, 2015 Release) - B...",5.0
4,Amazon - Amazon Tap Portable Bluetooth and Wi-...,4.5


## 4. Learning the similarity weights (Pearson Correlation)

#### The users who have seen the same movies
With the movies ID's in our input, we can get the subset of users who have watched and reviewed these movies in our input.


In [ ]:
#Filtering out users who have watched movies that the input has watched
userSubset = product_df[product_df['name'].isin(inputProducts['name'].tolist())]
userSubset

,name,reviews.rating,reviews.username
0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,Adapter
1,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,truman
2,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,DaveZ
3,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",4.0,Shacks
4,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,explore42
...,...,...,...
25346,Amazon - Amazon Tap Portable Bluetooth and Wi-...,5.0,Bmschroe
25347,Amazon - Amazon Tap Portable Bluetooth and Wi-...,5.0,Funtulips
25348,Amazon - Amazon Tap Portable Bluetooth and Wi-...,4.0,Marko3d
25349,Amazon - Amazon Tap Portable Bluetooth and Wi-...,5.0,Gdogg


We now group up the rows by user ID.

In [ ]:
userSubsetGroup = userSubset.groupby(['reviews.username'])

Lets look at one of the users, e.g. the one with userID=1130

In [ ]:
userSubsetGroup.get_group('Chris')

,name,reviews.rating,reviews.username
44,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,Chris
3502,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",5.0,Chris
17550,Amazon Kindle Paperwhite - eBook reader - 4 GB...,5.0,Chris


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [ ]:
#Sorting it so users with movies most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [ ]:
#Have a look at first user
userSubsetGroup[1]

('Andy',
                                                     name  ...  reviews.username
 72     All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...  ...              Andy
 3617   Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...  ...              Andy
 19137  Amazon Kindle Paperwhite - eBook reader - 4 GB...  ...              Andy
 25105  Amazon - Amazon Tap Portable Bluetooth and Wi-...  ...              Andy
 
 [4 rows x 3 columns])

#### Similarity of users to input user
Next, we are going to compare all users to our specified user and find the group of users that is most similar.  
we're going to find out how similar each user is to the input through the __Pearson Correlation__. It is used to measure the strength of a linear association between two variables.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y, then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales.

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

__Pearson Correlation Coeff__ vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [ ]:
#We will select a subset of users to iterate through, to not waste too much time going through every single user.
userSubsetGroup = userSubsetGroup[0:200]

In [ ]:
for name, group in userSubsetGroup:
    print(name)
    print(group)
    break

Alex
                                                    name  ...  reviews.username
1364   All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...  ...              Alex
5575   Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...  ...              Alex
17354  Amazon Kindle Paperwhite - eBook reader - 4 GB...  ...              Alex
25327  Amazon - Amazon Tap Portable Bluetooth and Wi-...  ...              Alex

[4 rows x 3 columns]


In [ ]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Sorting the input to match current user group so the values aren't mixed up later on
    group = group.sort_values(by='name')
    #Get the review scores for the movies that they both have in common
    temp_df = inputProducts[inputProducts['name'].isin(group['name'].tolist())]
    #Store them in a temporary variable in a list format
    tempRatingList = temp_df['rating'].tolist()
    #Also put the current user group reviews in a list format
    tempGroupList = group['reviews.rating'].tolist()
    # print(temp_df,group)
    #Pearson correlation coefficient and the p-value for testing non-correlation
    pearsonCorrelationDict[name] = round(pearsonr(tempRatingList, tempGroupList)[0], 3)
      

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [ ]:
pearsonCorrelationDict.items()

dict_items([('Alex', 0.436), ('Andy', 0.126), ('Anonymous', nan), ('Bobby', -0.218), ('Craig', -0.218), ('Dave', 0.309), ('Eric', -0.63), ('Jess', nan), ('JoJo', nan), ('Joey', 0.882), ('John', -0.873), ('Joseph', 0.126), ('Kathy', nan), ('Kindle', -0.522), ('Matt', -0.63), ('Mike', -0.309), ('Paul', -0.63), ('Rich', -0.873), ('Stacy', 0.436), ('dave', -0.63), ('1234', -1.0), ('Aaron', nan), ('Angie', nan), ('Annie', 0.866), ('April', nan), ('Barb', nan), ('Benjamin', -0.866), ('Bernie', nan), ('Bill', 0.866), ('Brad', -0.866), ('Brandon', -0.866), ('Brian', 0.866), ('Bubba', nan), ('Buddy', nan), ('Butch', 0.0), ('Carlos', -0.756), ('Cathy', 0.0), ('Charles', nan), ('Charlie', nan), ('Chris', nan), ('Christine', nan), ('Cindy', 0.866), ('Coco', -0.0), ('Connie', nan), ('Cowboy', -0.189), ('Crystal', nan), ('Dani', -0.866), ('Daniel', -0.803), ('Danny', 0.866), ('David', 0.866)])

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index').reset_index(drop=False) #orient: key is index or column
pearsonDF.columns = ['userId', 'similarityIndex']
pearsonDF = pearsonDF.sort_values(by='similarityIndex', ascending=False)
pearsonDF

,userId,similarityIndex
171,Aluisi,1.0
184,Apple,1.0
164,Aimee,1.0
166,AlexG,1.0
168,Alicia,1.0
...,...,...
188,Ashi,NaN
189,Ashley,NaN
193,Awesome,NaN
198,Barbara,NaN


#### The top x similar users to input user
Now let's get the top 20 users who are most similar to the input.

In [ ]:
topUsers = pearsonDF.iloc[0:20]
topUsers

,userId,similarityIndex
171,Aluisi,1.000
184,Apple,1.000
164,Aimee,1.000
166,AlexG,1.000
168,Alicia,1.000
174,Anderson,1.000
175,Andrea,1.000
159,AAM1,1.000
181,Anne,1.000
186,Arthur,1.000


## 5. Find the recommendations

#### Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our __pearsonDF__ from the ratings dataframe and then store their correlation in a new column called _similarityIndex".

In [ ]:
topUsersRating = pd.merge(topUsers, product_df, left_on='userId', right_on='reviews.username', how='inner')
topUsersRating

,userId,similarityIndex,name,reviews.rating,reviews.username
0,Aluisi,1.000,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",4.0,Aluisi
1,Aluisi,1.000,Amazon Kindle Paperwhite - eBook reader - 4 GB...,5.0,Aluisi
2,Apple,1.000,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,Apple
3,Apple,1.000,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",4.0,Apple
4,Apple,1.000,"Echo (White),,,\r\nEcho (White),,,",5.0,Apple
...,...,...,...,...,...
73,Matthew,0.866,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",3.0,Matthew
74,Matthew,0.866,Amazon Kindle Paperwhite - eBook reader - 4 GB...,4.0,Matthew
75,Matthew,0.866,"Kindle Voyage E-reader, 6 High-Resolution Disp...",5.0,Matthew
76,Matthew,0.866,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",4.0,Matthew


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [ ]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['reviews.rating']
topUsersRating.head()

,userId,similarityIndex,name,reviews.rating,reviews.username,weightedRating
0,Aluisi,1.0,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",4.0,Aluisi,4.0
1,Aluisi,1.0,Amazon Kindle Paperwhite - eBook reader - 4 GB...,5.0,Aluisi,5.0
2,Apple,1.0,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",5.0,Apple,5.0
3,Apple,1.0,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",4.0,Apple,4.0
4,Apple,1.0,"Echo (White),,,\r\nEcho (White),,,",5.0,Apple,5.0


In [ ]:
#Applies a sum to the topUsers after grouping it up by name
tempTopUsersRating = topUsersRating.groupby('name').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
name,,
0,9.614,44.070
"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta",10.480,51.534
"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 32 GB - Includes Special Offers, Magenta",1.000,5.000
"Amazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,\r\nAmazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,",1.800,9.000
"Amazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release), Cayenne Red,,,\r\nAmazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release), Cayenne Red,,,",0.882,4.410


In [ ]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['name'] = tempTopUsersRating.index
recommendation_df

,weighted average recommendation score,name
name,,
0,4.583940,0
"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta",4.917366,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 32 GB - Includes Special Offers, Magenta",5.000000,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
"Amazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,\r\nAmazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,",5.000000,Amazon - Amazon Tap Portable Bluetooth and Wi-...
"Amazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release), Cayenne Red,,,\r\nAmazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release), Cayenne Red,,,",5.000000,Amazon Fire Hd 6 Standing Protective Case(4th ...
"Amazon Fire Tv,,,\r\nAmazon Fire Tv,,,",4.789385,"Amazon Fire Tv,,,\r\nAmazon Fire Tv,,,"
"Amazon Kindle Paperwhite - eBook reader - 4 GB - 6 monochrome Paperwhite - touchscreen - Wi-Fi - black,,,",4.784965,Amazon Kindle Paperwhite - eBook reader - 4 GB...
"Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue,,,",5.000000,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...
"Echo (White),,,\r\nEcho (White),,,",5.000000,"Echo (White),,,\r\nEcho (White),,,"


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df

,weighted average recommendation score,name
name,,
"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 32 GB - Includes Special Offers, Magenta",5.000000,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."
"Amazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,\r\nAmazon - Amazon Tap Portable Bluetooth and Wi-Fi Speaker - Black,,,",5.000000,Amazon - Amazon Tap Portable Bluetooth and Wi-...
"Amazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release), Cayenne Red,,,\r\nAmazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release), Cayenne Red,,,",5.000000,Amazon Fire Hd 6 Standing Protective Case(4th ...
"Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue,,,",5.000000,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...
"Echo (White),,,\r\nEcho (White),,,",5.000000,"Echo (White),,,\r\nEcho (White),,,"
"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Green Kid-Proof Case",5.000000,"Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16..."
"Kindle Paperwhite,,,\r\nKindle Paperwhite,,,",5.000000,"Kindle Paperwhite,,,\r\nKindle Paperwhite,,,"
"Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers,",5.000000,"Kindle Voyage E-reader, 6 High-Resolution Disp..."
"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Magenta",4.917366,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,..."


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages
* Takes other user's ratings into consideration
* Doesn't need to study or extract information from the recommended item
* Adapts to the user's interests which might change over time

##### Disadvantages
* Approximation function can be slow
* There might be a low of amount of users to approximate
* Privacy issues when trying to learn the user's preferences